In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing, metrics
tf.get_logger().setLevel('ERROR')

In [ ]:
def create_bert_model(bert_enc, bert_prepr, classes_count):
    '''
    Create model based on BERT.
    Parameters:
        bert_enc (str): Link to BERT model.
        bert_prepr (str): Link to BERT preprocessing.
        classes_count (int): Number of classes (genres).
    Returns:
        Model: Created model.
    '''
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    preprocessing_layer = hub.KerasLayer(bert_prepr, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(bert_enc, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(classes_count, activation='softmax', name='classifier')(net)
    model = tf.keras.Model(text_input, net)
    
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
def transform_X(data_x):
    '''
    Transform Series into numpy array.
    Parameters:
        data_x (Series): Lyrics of the data.
    Returns:
        ndarray: Array with lyrics.
    '''
    X = np.array(data_x)
    X = np.asarray(X).astype('str')
    X = X.reshape(-1,1)
    return X

In [ ]:
def train_bert(data_x, data_y, test_x, test_y, model, label_encoder, dataset_name, epochs=1, model_dir='models', filename='bert_tuned'):
    '''
    Train a model.
    Parameters:
        data_x (Series): Lyrics of the training data.
        data_y (Series): Genres of the training data.
        test_x (Series): Lyrics of the validation/test data.
        test_y (Series): Genres of the validation/test data.
        model (Model): Created BERT model.
        label_encoder (LabelEncoder): Label encoder after fitting.
        dataset_name (str): Name of the dataset.
        epochs (int): Number of epochs.
        model_dir (str): Name of directory for saved models.
        filename (str): Part of the saved model file name.
    Returns:
        History: A History object.
    '''
    print('Training...')
    fname = os.path.join(model_dir, dataset_name, f'model_{filename}')
    Y = label_encoder.transform(data_y).reshape(-1,1)
    X = transform_X(data_x)
    Yt = label_encoder.transform(test_y).reshape(-1,1)
    Xt = transform_X(test_x)
    history = model.fit(x=X, y=Y, epochs=epochs, validation_data=(Xt, Yt))
    model.save(fname)
    print('Success!')
    return history

In [ ]:
def test_bert(data_x, model, label_encoder, dataset_name, pred_dir='predictions', filename='bert_tuned'):
    '''
    Test a model and return predictions.
    Parameters:
        data_x (Series): Lyrics of the test data.
        model (Model): Trained BERT model.
        label_encoder (LabelEncoder): Label encoder after fitting.
        dataset_name (str): Name of the dataset.
        pred_dir (str): Name of directory for saved predictions.
        filename (str): Part of the saved predictions file name.
    Returns:
        ndarray: Predicted genres.
    '''
    print('Testing...')
    fname = os.path.join(pred_dir, dataset_name, f'model_{filename}.csv')
    if os.path.exists(fname):
        os.remove(fname)
    X = transform_X(data_x)
    predictions_enc = np.argmax(model.predict(X), axis=1).flatten()
    predictions = label_encoder.inverse_transform(predictions_enc)
    pd.DataFrame(predictions.reshape(-1, 1)).to_csv(fname, mode='a', index=False, header=False)
    print('Success!')
    return predictions

In [ ]:
def get_results(y_true, y_pred):
    '''
    Print accuracy, balanced accuracy and F1 score.
    Parameters:
        y_true (Series): True genres of the test data.
        y_pred (Series): Predicted genres of the test data.
    '''
    print('RESULTS:')
    print(f'accuracy = {metrics.accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'balanced accuracy = {metrics.balanced_accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'f1 score = {metrics.f1_score(y_true=y_true, y_pred=y_pred, average='weighted')}')

In [ ]:
# Parameters

dataset_name = 'small_balanced'

In [ ]:
# Links to models

# bert_en_uncased_L-12_H-768_A-12
bert_enc = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_prepr = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
# small_bert/bert_en_uncased_L-2_H-256_A-4
bert_enc_sm = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1'
bert_prepr_sm = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
# Creation of necessary directories

model_dir = os.path.join('models', dataset_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

pred_dir = os.path.join('predictions', dataset_name)
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

In [ ]:
# Reading training and test data from CSV files

train_data = pd.read_csv(f'data/train/{dataset_name}.csv')
test_data = pd.read_csv(f'data/test/{dataset_name}.csv')

In [ ]:
# Label encoder fitting

genres = np.unique(train_data.genre)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(genres)
label_encoder.classes_

In [ ]:
# Creation of Smaller BERT model

bert_model_sm = create_bert_model(bert_enc_sm, bert_prepr_sm, len(label_encoder.classes_))

In [ ]:
# Creation of BERT model

bert_model = create_bert_model(bert_enc, bert_prepr, len(label_encoder.classes_))

In [ ]:
# Training of Smaller BERT model

hist_sm = train_bert(train_data.lyrics, train_data.genre, 
                     test_data.lyrics, test_data.genre,
                     bert_model_sm, label_encoder, dataset_name, epochs=1, filename='small_bert_tuned')

In [ ]:
# Testing of Smaller BERT model and printing results

preds = test_bert(test_data.lyrics, bert_model_sm, label_encoder, dataset_name, filename='small_bert_tuned')
get_results(test_data.genre, preds)

In [ ]:
# Training of BERT model

hist = train_bert(train_data.lyrics, train_data.genre, 
                  test_data.lyrics, test_data.genre,
                  bert_model, label_encoder, dataset_name, epochs=1)

In [ ]:
# Testing of BERT model and printing results

preds = test_bert(test_data.lyrics, bert_model, label_encoder, dataset_name)
get_results(test_data.genre, preds)